In [15]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import pickle

from agent import DRLAgent

from meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from meta.preprocessor.preprocessors import FeatureEngineer
from meta.preprocessor.preprocessors import data_split
from agent import *
from meta.preprocessor.yahoodownloader import YahooDownloader
import meta.config as config
from ppo import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
startingDate = '2012-1-1'
endingDate = '2020-1-1'
splitingDate = '2018-1-1'

df_price = YahooDownloader(startingDate, endingDate, ['AAPL']).fetch_data()
display(df_price)
df = df_price[['date', 'open', 'close', 'high', 'low', 'adjcp', 'volume', 'tic']]
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['day'] = [x.weekday() for x in df['date']]
df.drop_duplicates(['tic', 'date'], inplace=True)

trade_date = df['date'].to_numpy()


df_dict = {'trade_date':[], 'tic':[], 'weights':[]}
testing_window = pd.Timedelta(np.timedelta64(2,'Y'))
max_rolling_window = pd.Timedelta(np.timedelta64(10, 'Y'))

start_date = np.datetime64('2012-01-01')
end_date = np.datetime64('2020-01-01')
split_date = np.datetime64('2018-01-01')

for idx in range(1, len(trade_date)):
    p1_stock = ['AAPL']
    earliest_date = pd.to_datetime(trade_date[idx-1]) # pd.to_datetime(trade_date[idx-1]) - max_rolling_window

    df_ = df[df['tic'].isin(p1_stock) & (df['date'] >= earliest_date) & (df['date'] < trade_date[idx])]
    print(df_)
    fe = FeatureEngineer(use_technical_indicator=True, use_turbulence=False, user_defined_feature=False)
    df_ = fe.preprocess_data(df_)
    df_ = df_.sort_values(['date', 'tic'], ignore_index=True)
    df_.index = df_.date.factorize()[0]
    cov_list = []
    return_list = []

    # look back is one year
    lookback = 252
    for i in range(lookback, len(df_.index.unique())):
        data_lookback = df_.loc[i-lookback:i,:]
        price_lookback = data_lookback.pivot_table(index='date', columns='tic', values='close')
        return_lookback = price_lookback.pct_change().dropna()
        return_list.append(return_lookback)
        covs = return_lookback.cov().values
        cov_list.append(covs)

    df_cov = pd.DataFrame({'date':df_.date.unique()[lookback:], 'cov_list':cov_list, 'return_list':return_list})
    df_ = df_.merge(df_cov, on='date')
    df_ = df_.sort_values(['date','tic']).reset_index(drop=True)

    stock_dimension = len(df_.tic.unique())
    state_space = stock_dimension
    env_kwargs = {
        "hmax": 100, 
        "initial_amount": 1000000, 
        "transaction_cost_pct": 0.001, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4
    }

    ppo_model, ppo_return = run_ppo_model(df_, "date", pd.to_datetime(trade_date[idx-1]), env_kwargs, 
                                          start_date, end_date, split_date)
                                          # testing_window, max_rolling_window)
    
    trade = data_split(df_, pd.to_datetime(trade_date[idx-1]), pd.to_datetime(trade_date[idx]))
    e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)
    df_daily_return, df_actions = DRLAgent.DRL_prediction(model=ppo_model, environment=e_trade_gym)

    for i in range(len(df_actions)):
        for j in df_actions.columns:
            df_dict['trade_date'].append(df_actions.index[i])
            df_dict['tic'].append(j)
            df_dict['weights'].append(df_actions.loc[df_actions.index[i], j])


df_rl = pd.DataFrame(df_dict)
df_rl.to_csv("drl_weight.csv")

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2012, 9)


,date,open,high,low,close,adjcp,volume,tic,day
0,2012-01-03,14.621429,14.732143,14.607143,14.686786,12.500191,302220800,AAPL,1
1,2012-01-04,14.642857,14.810000,14.617143,14.765714,12.567370,260022000,AAPL,2
2,2012-01-05,14.819643,14.948214,14.738214,14.929643,12.706893,271269600,AAPL,3
3,2012-01-06,14.991786,15.098214,14.972143,15.085714,12.839726,318292800,AAPL,4
4,2012-01-09,15.196429,15.276786,15.048214,15.061786,12.819365,394024400,AAPL,0
...,...,...,...,...,...,...,...,...,...
2007,2019-12-24,71.172501,71.222504,70.730003,71.067497,69.517082,48478800,AAPL,1
2008,2019-12-26,71.205002,72.495003,71.175003,72.477501,70.896339,93121200,AAPL,3
2009,2019-12-27,72.779999,73.492500,72.029999,72.449997,70.869415,146266000,AAPL,4
2010,2019-12-30,72.364998,73.172501,71.305000,72.879997,71.290054,144114400,AAPL,0


        date       open      close       high        low      adjcp  \
0 2012-01-03  14.621429  14.686786  14.732143  14.607143  12.500191   

      volume   tic  day  
0  302220800  AAPL    1  
Successfully added technical indicators
Empty DataFrame
Columns: [date, open, close, high, low, adjcp, volume, tic, day, macd, boll_ub, boll_lb, rsi_30, cci_30, dx_30, close_30_sma, close_60_sma, cov_list, return_list]
Index: []


D:\Users\amine\AppData\Local\Temp\ipykernel_17884\383091794.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


KeyError: 0